# Modelo 3: LSTM(Bilstm) + CRF + embedding

Realizado por:
- Jose Luis Hincapie Bucheli (2125340)
- Sebastián Idrobo Avirama (2122637)
- Paul Rodrigo Rojas Guerrero (2127891)

---

# Traer información desde Google Drive

Si te encuentras usando Google Colab, probablemente necesites descomentar los siguientes bloques de código y modificar las rutas para que funcione con el tuyo:

In [49]:
# Borrar esto
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import sys
import os

# sys.path.append('/content/drive/My Drive/packages') # Poner Rutas de los paquetes aquí
# sys.path.append('/content/drive/My Drive/embeddings') # Poner Rutas de los embeddings aquí

# path = '/content/drive/My Drive/packages'

# Se verifica que todo se encuentre funcionando correctamente
# files = os.listdir(path)
# print(files)

['wrapper.py', 'tf2crf.py', '__pycache__', 'utils.py', 'crfta.py']


---

# Instalación e importación de paquetes

In [51]:
try:
    import seqeval
except ModuleNotFoundError as err:
    !pip install seqeval

In [52]:
!pip install tensorflow-addons==.0.23.0
!pip install tensorflow==2.15.0
!pip install keras==2.15.0
!pip install nltk==3.8.1
!pip install seqeval
!pip install gensim

In [53]:
import sys
import os

# Add the directory to the system path
sys.path.append('/packages')

In [54]:
import tensorflow as tf
#matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [55]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from itertools import islice

#from tabulate import tabulate
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report as eskclarep
#from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score
#from seqeval.metrics import classification_report as seqclarep
from sklearn.preprocessing import LabelBinarizer
from itertools import chain

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Concatenate, Lambda, Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, InputLayer, Activation, Flatten
from tensorflow.keras.optimizers import Adam, schedules
from crfta import CRF as crf4
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

from IPython.core.display import display, HTML

import datetime, os
import random
import seqeval

# Importación de dataset conll2002

In [56]:
import nltk
nltk.download('conll2002')
nltk.corpus.conll2002.fileids()

[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [57]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
eval_sents = list(nltk.corpus.conll2002.iob_sents('esp.testa'))
print(len(train_sents),len(max(train_sents,key=len)))
print(len(test_sents),len(max(test_sents,key=len)))
print(len(eval_sents),len(max(eval_sents,key=len)))

8323 1238
1517 202
1915 141


In [58]:
print(train_sents[0])

[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


# Parte 1: Preprocesamiento de los datos

En esta parte, los datos que serán utilizados -conll2002- deben de ser procesados de tal manera que los tokens y labels sean separados y accedidos por medio de listas de índices y convertir las sentencias de representaciones textuales a secuencias de índices.

In [59]:
def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [60]:
X_train = [sent2tokens(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2tokens(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

X_eval = [sent2tokens(s) for s in eval_sents]
y_eval = [sent2labels(s) for s in eval_sents]

In [61]:
print(X_train[2])
print(y_train[2])

['El', 'Abogado', 'General', 'del', 'Estado', ',', 'Daryl', 'Williams', ',', 'subrayó', 'hoy', 'la', 'necesidad', 'de', 'tomar', 'medidas', 'para', 'proteger', 'al', 'sistema', 'judicial', 'australiano', 'frente', 'a', 'una', 'página', 'de', 'internet', 'que', 'imposibilita', 'el', 'cumplimiento', 'de', 'los', 'principios', 'básicos', 'de', 'la', 'Ley', '.']
['O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O']


In [62]:
import numpy as np

words, tagsss = set([]), set([])

for s in (X_train + X_eval + X_test):
    for w in s:
        words.add(w.lower())

for ts in (y_train + y_eval + y_test):
    for t in ts:
        tagsss.add(t)

word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs

tag2index = {t: i + 2 for i, t in enumerate(list(tagsss))}
tag2index['-PAD-'] = 0  # The special value used to padding
tag2index['-OOV-'] = 1  # The special value used to padding

print (len(word2index))
print (len(tag2index))
print(tag2index)


print(tagsss)

28384
11
{'I-PER': 2, 'I-MISC': 3, 'B-ORG': 4, 'B-MISC': 5, 'B-LOC': 6, 'I-ORG': 7, 'I-LOC': 8, 'B-PER': 9, 'O': 10, '-PAD-': 0, '-OOV-': 1}
{'I-PER', 'I-MISC', 'B-ORG', 'B-MISC', 'B-LOC', 'I-ORG', 'I-LOC', 'B-PER', 'O'}


In [63]:
train_sentences_X, eval_sentences_X, test_sentences_X, train_tags_y, eval_tags_y, test_tags_y = [], [], [], [], [], []

for s in X_train:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    train_sentences_X.append(s_int)

for s in X_eval:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    eval_sentences_X.append(s_int)

for s in X_test:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    test_sentences_X.append(s_int)

for s in y_train:
    s_int = []
    for w in s:
        try:
            s_int.append(tag2index[w])
        except KeyError:
            s_int.append(tag2index['-OOV-'])

    train_tags_y.append(s_int)

for s in y_eval:
    s_int = []
    for w in s:
        try:
            s_int.append(tag2index[w])
        except KeyError:
            s_int.append(tag2index['-OOV-'])

    eval_tags_y.append(s_int)

for s in y_test:
    s_int = []
    for w in s:
        try:
            s_int.append(tag2index[w])
        except KeyError:
            s_int.append(tag2index['-OOV-'])

    test_tags_y.append(s_int)


In [64]:
print("Longitudes de las Matrices:")
print(len(train_sentences_X))
print(len(eval_sentences_X))
print(len( test_sentences_X))
print(len(train_tags_y))
print(len(eval_tags_y))
print(len(test_tags_y))

print("\nMuestra de Datos presentes en las Matrices con las transformaciones:\n")


print(train_sentences_X[0])
print(eval_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(eval_tags_y[0])
print(test_tags_y[0])


Longitudes de las Matrices:
8323
1915
1517
8323
1915
1517

Muestra de Datos presentes en las Matrices con las transformaciones:

[1946, 16702, 22081, 28086, 1222, 18613, 6880, 16702, 26598, 28086, 1764]
[10486, 10258, 16702, 3585, 28086, 1222, 13000, 6880, 16702, 13584, 28086, 1764]
[10837, 24702, 1222, 13000, 6880, 16702, 13584, 28086, 1764]
[6, 10, 6, 10, 10, 10, 10, 10, 4, 10, 10]
[6, 8, 10, 6, 10, 10, 10, 10, 10, 4, 10, 10]
[6, 8, 10, 10, 10, 10, 4, 10, 10]


Una vez con estas variables, se debe de normalizar las matrices a partir de MAX_LENGTH, requiriendo de un array estático para servir como input de la RNN.

In [65]:

MAX_LENGTH=202
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
eval_sentences_X = pad_sequences(eval_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
eval_tags_y = pad_sequences(eval_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')

print(train_sentences_X[0])
print(train_sentences_X.shape)
print(eval_sentences_X[0])
print(eval_sentences_X.shape)
print(test_sentences_X[0])
print(test_sentences_X.shape)
print(train_tags_y[0])
print(train_tags_y.shape)
print(eval_tags_y[0])
print(eval_tags_y.shape)
print(test_tags_y[0])
print(test_tags_y.shape)



[ 1946 16702 22081 28086  1222 18613  6880 16702 26598 28086  1764     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

Y una vez normalizadas las matrices, se procede a representar las listas de indices de etiquetas a una representación one-hot para el procesamiento con el RNN

In [66]:
def to_categoricals(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [67]:
def encode(data):
    print('Shape of data (BEFORE encode): %s' % str(data.shape))
    encoded = to_categorical(data)
    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
    return encoded

In [68]:
cat_train_tags_y = to_categoricals(train_tags_y, len(tag2index))
cat_eval_tags_y  = to_categoricals(eval_tags_y, len(tag2index))
cat_test_tags_y  = to_categoricals(test_tags_y, len(tag2index))

print(cat_train_tags_y[1])
print(len(cat_train_tags_y))
print(cat_train_tags_y.shape)
print(len(cat_test_tags_y))

[[0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
8323
(8323, 202, 11)
1517


# Parte 2: Entrenamiento del modelo de red

Se define la red neuronal para el trabajo de NER usando el modelo LSTM(Bilstm) + CRF + embedding, importando los módulos necesarios para el mismo y también parametrizando la RNN a entrenar.
- Se trae el embedding con tamaño de 300.
- Se genera el modelo LSTM(Bilstm) + CRF
- Se genera una capa de masking para que ignore los valores de 0

Una vez hecho esto, se procede a entrenar la red neuronal y a testear los resultados de la misma

In [69]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
EMBED_DIM = 300

embedding_file = "/content/drive/My Drive/embeddings/word2vex_jose_sebastian_paul300.txt" # Poner ruta del embedding a utilizar

embedding_matrix = bme(embedding_file, len(word2index), EMBED_DIM, word2index)

Cargando archivo...


0it [00:00, ?it/s]

Encontrado 220714 Word Vectors.


  0%|          | 0/28384 [00:00<?, ?it/s]

Convertidos: 17841 Tokens | Perdidos: 10543 Tokens


In [70]:
# TO-DO: No se ha hecho nada diferente por el momento

from tf2crf import CRF as crf6
from wrapper import ModelWithCRFLoss, ModelWithCRFLossDSCLoss
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
from tensorflow.keras.layers import Concatenate, Lambda, Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, InputLayer, Activation, Flatten, Masking
from tensorflow.keras.optimizers import Adam, schedules
input = Input(shape=(MAX_LENGTH,))
word_embedding_size = 300
EMBED_DIM=300
# Embedding Layer
model = Embedding(input_dim=len(word2index),
                weights=[embedding_matrix],
                output_dim=word_embedding_size,
                input_length=MAX_LENGTH,
                mask_zero=False)(input)

# BI-LSTM Layer
model = Bidirectional(LSTM(units=50,
                     return_sequences=True,
                     dropout=0.5,
                     recurrent_dropout=0.5))(model)

# LSTM(Bilstm) Layer
model = LSTM(units=50,
                     return_sequences=True,
                     dropout=0.5,
                     recurrent_dropout=0.5)(model)

model  = Dropout(0.5, name='dropout_lstm')(model)
model  = Dense(units=EMBED_DIM * 2, activation='tanh')(model)
model  = Dense(units=len(tag2index), activation='tanh')(model)

model  = Masking(mask_value=0.,input_shape=(MAX_LENGTH, len(tag2index)))(model)

crf = crf6(units=len(tag2index), name="ner_crf")
predictions = crf(model)

base_model = Model(inputs=input, outputs=predictions)
model = ModelWithCRFLoss(base_model, sparse_target=True)

model.compile(optimizer='adam')
#model.summary()

In [71]:
history= model.fit(train_sentences_X, cat_train_tags_y,
                       validation_data=(eval_sentences_X, cat_eval_tags_y),
                       batch_size=128,
                       epochs=50,
                       verbose=2)

Epoch 1/50
66/66 - 51s - loss: 142.1511 - accuracy: 0.9170 - val_loss_val: 61.9691 - val_val_accuracy: 0.9768 - 51s/epoch - 768ms/step
Epoch 2/50
66/66 - 38s - loss: 45.9855 - accuracy: 0.9783 - val_loss_val: 35.4018 - val_val_accuracy: 0.9792 - 38s/epoch - 582ms/step
Epoch 3/50
66/66 - 38s - loss: 31.2426 - accuracy: 0.9801 - val_loss_val: 27.9765 - val_val_accuracy: 0.9802 - 38s/epoch - 581ms/step
Epoch 4/50
66/66 - 38s - loss: 26.5557 - accuracy: 0.9805 - val_loss_val: 24.7721 - val_val_accuracy: 0.9803 - 38s/epoch - 582ms/step
Epoch 5/50
66/66 - 39s - loss: 23.8864 - accuracy: 0.9806 - val_loss_val: 22.9194 - val_val_accuracy: 0.9804 - 39s/epoch - 584ms/step
Epoch 6/50
66/66 - 38s - loss: 22.7457 - accuracy: 0.9806 - val_loss_val: 21.6754 - val_val_accuracy: 0.9804 - 38s/epoch - 581ms/step
Epoch 7/50
66/66 - 38s - loss: 21.4890 - accuracy: 0.9806 - val_loss_val: 20.7350 - val_val_accuracy: 0.9804 - 38s/epoch - 581ms/step
Epoch 8/50
66/66 - 38s - loss: 20.9109 - accuracy: 0.9806 - v

In [72]:
print(tag2index)
print(test_sentences_X)
y_pred= model.predict(test_sentences_X)
print(y_pred.shape)
print(y_pred)

{'I-PER': 2, 'I-MISC': 3, 'B-ORG': 4, 'B-MISC': 5, 'B-LOC': 6, 'I-ORG': 7, 'I-LOC': 8, 'B-PER': 9, 'O': 10, '-PAD-': 0, '-OOV-': 1}
[[10837 24702  1222 ...     0     0     0]
 [18846     0     0 ...     0     0     0]
 [22744 28127  9869 ...     0     0     0]
 ...
 [10837  8063 13650 ...     0     0     0]
 [22003 26309 20771 ...     0     0     0]
 [10837 28294 13201 ...     0     0     0]]
48/48 [==============================] - 5s 79ms/step
(1517, 202)
[[ 6  8 10 ...  0  0  0]
 [10  0  0 ...  0  0  0]
 [10 10 10 ...  0  0  0]
 ...
 [10  6 10 ...  0  0  0]
 [10 10 10 ...  0  0  0]
 [10 10 10 ...  0  0  0]]


In [73]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
index2tag = {i: t for t, i in tag2index.items()}
print(index2tag)
y1_pred = logits_to_tokens(y_pred, index2tag)
print(y1_pred[10])

{2: 'I-PER', 3: 'I-MISC', 4: 'B-ORG', 5: 'B-MISC', 6: 'B-LOC', 7: 'I-ORG', 8: 'I-LOC', 9: 'B-PER', 10: 'O', 0: '-PAD-', 1: '-OOV-'}
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PA

In [74]:
#print(Y_test[4])
print(test_tags_y.shape)

(1517, 202)


In [75]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
index2tag = {i: t for t, i in tag2index.items()}
print(index2tag)
y1_true = logits_to_tokens(test_tags_y, index2tag)
print(y1_true[10])

{2: 'I-PER', 3: 'I-MISC', 4: 'B-ORG', 5: 'B-MISC', 6: 'B-LOC', 7: 'I-ORG', 8: 'I-LOC', 9: 'B-PER', 10: 'O', 0: '-PAD-', 1: '-OOV-'}
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-'

## Calidad obtenida del modelo

Se evalua la calidad de los resultados del modelo usando las siguientes métricas: *precision, recall, accuracy* y *F1-score*.

In [76]:
from seqeval.metrics import classification_report as seqclarep
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score
print("precision: {:.1%}".format(precision_score(y1_true, y1_pred)))
print("   recall: {:.1%}".format(recall_score(y1_true,    y1_pred)))
print(" accuracy: {:.1%}".format(accuracy_score(y1_true,  y1_pred)))
print(" F1-score: {:.1%}".format(f1_score(y1_true,        y1_pred)))

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: -PAD- seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


precision: 68.8%
   recall: 67.8%
 accuracy: 99.1%
 F1-score: 68.3%


Se genera un reporte de la clasificación basada en las etiquetas predichas y las reales, esto brindado en un DataFrame.

In [77]:
import pandas as pd
li1 = sum(y1_true, [])
li2 = sum(y1_pred, [])

results = pd.DataFrame(columns=['Expected', 'Predicted'])

results['Expected'] = li1
results['Predicted'] = li2

In [78]:
from sklearn.metrics import classification_report as eskclarep
report = eskclarep(results['Expected'], results['Predicted'])

print(report_to_df(report))

  Class Name precision recall f1-score support
0      -PAD-      1.00   1.00     1.00  254901
1      B-LOC      0.74   0.77     0.75    1084
2     B-MISC      0.35   0.36     0.35     339
3      B-ORG      0.78   0.78     0.78    1400
4      B-PER      0.83   0.69     0.75     735
5      I-LOC      0.65   0.67     0.66     325
6     I-MISC      0.51   0.37     0.43     557
7      I-ORG      0.77   0.64     0.70    1104
8      I-PER      0.90   0.79     0.84     634
9          O      0.97   0.98     0.98   45355


Por último, se realizan evaluaciones del modelo a partir de algunas oraciones.

In [79]:
test_sample1 = [
    "James Rodriguez es el jugador colombiano más importante con Radamel Falcao.".split(),
    " Jugadores de la selección Colombia que juegan en el Reino Unido".split()
]

print(test_sample1)

test_sample2 = [
    "La federación Nacional de cafeteros de Colombia es dirigida por Horacio Sánchez".split(),
    " y se ubica en las ciudades de  Cali y Medellín con el instituto colombiano del café ".split()
]

print(test_sample2)

[['James', 'Rodriguez', 'es', 'el', 'jugador', 'colombiano', 'más', 'importante', 'con', 'Radamel', 'Falcao.'], ['Jugadores', 'de', 'la', 'selección', 'Colombia', 'que', 'juegan', 'en', 'el', 'Reino', 'Unido']]
[['La', 'federación', 'Nacional', 'de', 'cafeteros', 'de', 'Colombia', 'es', 'dirigida', 'por', 'Horacio', 'Sánchez'], ['y', 'se', 'ubica', 'en', 'las', 'ciudades', 'de', 'Cali', 'y', 'Medellín', 'con', 'el', 'instituto', 'colombiano', 'del', 'café']]


In [80]:
test_samples_X = []
for s in test_sample2:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)

test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)
print(test_samples_X.shape)

[[10837 20092 16965 16754 24347 16754 20270 19921 19723 20126     1 26892
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 

In [81]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

1/1 [==============================] - 0s 87ms/step
[[10  4  7  7  7  7  7 10 10 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [10 10 10 10 10 10 10  6 10 10 10 10 10 10 10 10  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  

In [82]:
#print(len(predictions))
log_tokens = logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})
print(log_tokens)

[['O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-

In [83]:
!pip install tabulate
from tabulate import tabulate

heads1 = test_sample2[0]
body1 = [log_tokens[0][:len(test_sample2[0])]]

heads2 = test_sample2[1]
body2 = [log_tokens[1][:len(test_sample2[1])]]

print(tabulate(body1, headers=heads1))

print ("\n")

print(tabulate(body2, headers=heads2))

## postagging Freeling 4.1

## El      hombre   bajo     corre    bajo  el      puente   con  bajo  índice   de  adrenalina  .
## DA0MS0  NCMS000  AQ0MS00  VMIP3S0  SP    DA0MS0  NCMS000  SP   SP    NCMS000  SP  NCFS000     Fp


## pos tagger Stanford NLP

## El      hombre   bajo     corre    bajo  el      puente   con    bajo   índice  de    adrenalina  .
## da0000  nc0s000  aq0000   vmip000  sp000 da0000  nc0s000  sp000  aq0000 nc0s000 sp000 nc0s000     fp

La    federación    Nacional    de     cafeteros    de     Colombia    es    dirigida    por    Horacio    Sánchez
----  ------------  ----------  -----  -----------  -----  ----------  ----  ----------  -----  ---------  ---------
O     B-ORG         I-ORG       I-ORG  I-ORG        I-ORG  I-ORG       O     O           O      -PAD-      -PAD-


y    se    ubica    en    las    ciudades    de    Cali    y    Medellín    con    el    instituto    colombiano    del    café
---  ----  -------  ----  -----  ----------  ----  ------  ---  ----------  -----  ----  -----------  ------------  -----  ------
O    O     O        O     O      O           O     B-LOC   O    O           O      O     O            O             O      O
